In [12]:
import subprocess
import os
import re

In [15]:
working_HGS_dir = r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs"
working_DSSAT_dir = r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat"

# Get a fixed list of relevant model names at the start
model_files = [
    re.match(r'(lys_e_day)(\d+)\.grok', f).groups()
    for f in os.listdir(working_HGS_dir)
    if re.match(r'lys_e_day\d+\.grok', f)
]
#sort the list of model files
sorted_model_names = [f"{prefix}{int(day)}" for prefix, day in sorted(model_files, key=lambda x: int(x[1]))]


In [19]:

# create bfx file for hgs
output_file = working_HGS_dir +'/batch.pfx'
with open(output_file, 'w') as f:
    f.write(sorted_model_names[0] + '\n')
#run grok and phgs subprocess
subprocess.run(['cmd', '/c', 'grok'], cwd=working_HGS_dir) #, stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
subprocess.run(['cmd', '/c', 'phgs'], cwd=working_HGS_dir) #stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
#check what is running and remove from the list
print(f"Processing model: {sorted_model_names[0]}")
# run create DRN function

# run all existing DSSAT models for HGS zones 
for DSSAT_model_name in os.listdir(working_DSSAT_dir):
    DSSAT_dict=working_DSSAT_dir + '\'' + DSSAT_model_name
    print(DSSAT_dict)
    #subprocess.run(['cmd', '/c', 'C:\Users\glogow0000\HGS-DSSAT\codebase\DSSAT\Coup\DSSAT\build\bin\Debug\dscsm048.exe'], cwd=(working_DSSAT_dir+"/"+DSSAT_model_name))
    
#open loop for 
for model_name in sorted_model_names[1:]:
    with open(output_file, 'w') as f:
        f.write(model_name + '\n')
    #while While Loop check if day is higher than final simulation day
    
    #while While Loop, check every 5 seconds for DSSAT completion
    
    #run grok and phgs subprocess
    subprocess.run(['cmd', '/c', 'grok'], cwd=working_HGS_dir) #, stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    subprocess.run(['cmd', '/c', 'phgs'], cwd=working_HGS_dir) #stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    print(f"Processing model: {model_name} ")

    #run create DRN function

Processing model: lys_e_day0
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat'1
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat'2
Processing model: lys_e_day1 
Processing model: lys_e_day2 
Processing model: lys_e_day3 
Processing model: lys_e_day4 
Processing model: lys_e_day5 
Processing model: lys_e_day6 
Processing model: lys_e_day7 
Processing model: lys_e_day8 
Processing model: lys_e_day9 
Processing model: lys_e_day10 
Processing model: lys_e_day11 
Processing model: lys_e_day12 
Processing model: lys_e_day13 
Processing model: lys_e_day14 
Processing model: lys_e_day15 
Processing model: lys_e_day16 
Processing model: lys_e_day17 
Processing model: lys_e_day18 
Processing model: lys_e_day19 
Processing model: lys_e_day20 
Processing model: lys_e_day21 
Processing model: lys_e_day22 
Processing model: lys_e_day23 
Processing model: lys_e_day24 
Processing model: lys_e_day25 
Processing model: lys_e_day26 
Processing model: lys_e_day27 
Processing model: lys_e_day28 
Pr